<a href="https://colab.research.google.com/github/YuvrajTak/CIPHERML_Assignments/blob/master/_Assignment_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing Libraries

In [ ]:
import pandas as pd;
import numpy as np;
import matplotlib.pyplot as plt;
import seaborn as sea;

# Importing DataSet

In [ ]:
df = pd.read_csv("/content/adult.csv");
df

**Observation**


*   This data set contain some missing values "?"


In [ ]:
missing_val = ['?']

df = pd.read_csv('/content/adult.csv', na_values = missing_val)

df.head()



In [ ]:
row = df.shape[0]
col = df.shape[1]
df.shape

**Observation**


    Number of Rows in Dataset = 48842
    Number of Columns in Dataset = 15


In [ ]:
df.info()

**Observation**

 6 continuous, 9 nominal attributes in data set

Continuous attributes

    age
    fnlwgt
    educational-num
    capital-gain
    capital-loss
    hours-per-week

nominal attributes

    workclass
    education
    material-status
    occupation
    relationship
    race
    gender
    native-country
    income




# Find Total Null Values in their columns

In [ ]:
df.isnull().sum()

**Observation**

Null value contain column are

    workclass 2799
    occuption 2809
    native-country 857



# Descriptive Analysis

In [ ]:
df.describe(include="all")



**Observation**

    age of most working people is 38.
    Most of people have their native country United-States(43832)
    Most of the peoples earn more then 50K (37155)
    Most of the people work in private sector (33906)
    Most number of peoples are high school graduate (15784)
    White peoples have the majority in employment.
    Most of working peoples are Male and married.
    Most peoples's have their occputation as Prof-specialty



# Handling Missing Values

In [ ]:
df.loc[df["workclass"].isnull(),["occupation","workclass","native-country"]]



**Observation**

when ever workclass column have null values occuption also have null values.


In [ ]:
df.loc[df["native-country"].isnull(),["workclass"]]

# Removing The Missing Values

In [ ]:
df.dropna(inplace=True)
df

In [ ]:
df.isnull().sum()

In [ ]:
df.head()

In [ ]:
((row - df.shape[0] )/ row) *100



**Observation**

    7.41% of Rows have removed to handle missing values.



In [ ]:
df

**Observation**

   

--- This data set is ready for Analysis operation.





# Exploratory Data Analysis

In this given link we get EDA in great detail:-

https://github.com/YuvrajTak/CIPHERML_Assignments/blob/master/Assignment_4.ipynb

# Target Column
"income"

In [ ]:
df['income'].astype('category').value_counts()

**Observation**
  Our Target column "income" have two category.
  1. High income
  2. Low income

In [ ]:
#We derive two array
# One for object base column
#second for numeric base column
df_obj = []
df_num = []
for i in df:
    if df[i].dtype == "object":
        df_obj.append(i)
    else:
        df_num.append(i)

In [ ]:
df["income"].replace(to_replace="<=50K",value=0,inplace=True)
df["income"].replace(to_replace=">50K",value=1,inplace=True)

**Observation**
  
   Our Target column is replace from Object to Numeric(0,1) category.

In [ ]:
df.dropna(inplace=True,axis=0)
df.drop("fnlwgt",inplace=True,axis=1)
df_num.remove("fnlwgt")
df_obj.remove("income")

**Observaion**

Removing Some unnecessory column

In [ ]:
obj_data = df[df_obj]
df.drop(df_obj,axis=1,inplace=True)
df.reset_index(inplace=True)
df

In [ ]:
obj_data

Observation

All Object base data column

In [ ]:
from sklearn.preprocessing import OneHotEncoder
onecoder = OneHotEncoder(drop='first', sparse=False)

obj_data = pd.DataFrame(onecoder.fit_transform(obj_data),columns=onecoder.get_feature_names(obj_data.columns))

obj_data

**Observation**

Our feature (columns) increase and become 90 


In [ ]:
df = pd.concat([df,obj_data],axis=1)
df.drop("index",inplace=True,axis=1)
df

**Observation**

All feature(Numeric+Object) are concetinate.

# Split data in Test and Train

In [ ]:
from sklearn.model_selection import train_test_split 
#Import sklearn libery to split data.
train_data,test_data=train_test_split(df,test_size=.25,random_state=10)

# Standardizing numerical data

**Standardizing** - It refers to rescaling by the minimum and range of the vector, to make all the elements lie between 0 and 1

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(train_data[df_num])
train_data[df_num] = scaler.transform(train_data[df_num])

x_train = train_data.drop("income",axis=1)
y_train = train_data.income

# Encoding Categorical feature

In [ ]:
from sklearn.linear_model import LogisticRegression
log = LogisticRegression(max_iter=250)
log.fit(x_train,y_train)

**Observation**

Fit data for training

In [ ]:
test_data[df_num] = scaler.transform(test_data[df_num])
x_test = test_data.drop("income",axis=1)
y_test = test_data.income

y_predict = log.predict(x_test)
y_predict


**Observation**

Predict y Using our train Model


In [ ]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix


In [ ]:
#accuracy_score(y_predict,y_test)
accuracy_score(y_predict,y_test)

In [176]:
print(classification_report(y_predict,y_test))

              precision    recall  f1-score   support

           0       0.93      0.87      0.90      9066
           1       0.59      0.75      0.66      2240

    accuracy                           0.85     11306
   macro avg       0.76      0.81      0.78     11306
weighted avg       0.87      0.85      0.86     11306



In [ ]:
print(confusion_matrix(y_predict,y_test))

# Recursive Feature Elimination(RFE) reducing the dimensionality.

In [ ]:
from sklearn.feature_selection import RFE

log_temp = LogisticRegression(max_iter=250)
rfe = RFE(log_temp,20) 
rfe = rfe.fit(x_train, y_train) 

In [ ]:
temp_df = pd.DataFrame({'Columns' : df.drop("income",axis=1), 'Included' : rfe.support_, 'Ranking' : rfe.ranking_})

temp_df.loc[temp_df["Included"] == True]

In [ ]:
rfe_column = x_train.columns[rfe.support_]
rfe_column

In [ ]:
from sklearn.linear_model import LogisticRegression
logrfe = LogisticRegression(max_iter=250)
logrfe.fit(x_train.loc[:,rfe_column], y_train)

In [ ]:
from sklearn.linear_model import LogisticRegression
logrfe = LogisticRegression(max_iter=250)
logrfe.fit(x_train.loc[:,rfe_column], y_train)

In [ ]:
y_rfe_predict = logrfe.predict(x_test.loc[:,rfe_column])

In [ ]:
accuracy_score(y_rfe_predict,y_test)

In [ ]:
print(classification_report(y_rfe_predict,y_test))

In [ ]:
print(confusion_matrix(y_rfe_predict,y_test))

In [ ]:
df

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
df[df_num] = sc.fit_transform(df.loc[:,df_num])
df


# PCA

The main idea of principal component analysis (PCA) is to reduce the dimensionality of a data set consisting of many variables correlated with each other, either heavily or lightly, while retaining the variation present in the dataset, up to the maximum extent.

In [ ]:
from sklearn.decomposition import PCA
pca = PCA()
pca.fit(df)

In [ ]:
np.cumsum(pca.explained_variance_ratio_)

In [ ]:
len(np.cumsum(pca.explained_variance_ratio_))

In [ ]:
sea.lineplot(np.arange(1,len(np.cumsum(pca.explained_variance_ratio_))+1),np.cumsum(pca.explained_variance_ratio_))
plt.show()

In [ ]:
plt.figure(figsize=(20,8))
sea.barplot(np.arange(1,len(np.cumsum(pca.explained_variance_ratio_))+1),np.cumsum(pca.explained_variance_ratio_))
plt.xticks(rotation=90)
plt.show()

**Observation**

Graph Show Most Of data is covered till 47 column

In [ ]:
labels = df.loc[:,"income"]
df.drop("income",axis=1,inplace=True)

In [ ]:
pca = PCA(n_components=47)
pca = pca.fit(df)
dataPCA = pca.transform(df)
dataPCA

**Observation**
PCA On feature n_components = 47
Data after PCA in array form

In [ ]:
#Importing all Required Libery for classification
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(dataPCA[:45222], labels, test_size=.25, random_state=2)

# Classification Report Based Upon Logistic Regression

In [ ]:
log_pca = LogisticRegression()
log_pca.fit(x_train,y_train)
y_predict = log_pca.predict(x_test)
print("Accuracy score ",accuracy_score(y_test,y_predict))
print("\n Confusion metricx ")
print(confusion_matrix(y_test,y_predict))
print("\n classification Details ")
print(classification_report(y_test,y_predict))

# Classification Report Based Upon Dicision Tree

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(x_train,y_train)
y_predict = dt.predict(x_test)
print("Accuracy score ",accuracy_score(y_test,y_predict))
print("\n Confusion metricx ")
print(confusion_matrix(y_test,y_predict))
print("\n classification Details ")
print(classification_report(y_test,y_predict))

# Classification Report Based Upon KNN 

In [ ]:
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(x_train,y_train)
y_predict = knn.predict(x_test)
print("Accuracy score ",accuracy_score(y_test,y_predict))
print("\n Confusion metricx ")
print(confusion_matrix(y_test,y_predict))
print("\n classification Details ")
print(classification_report(y_test,y_predict))

# Classification Report Based Upon Support Vector Machine

In [ ]:
sv = SVC()
sv.fit(x_train,y_train)
y_predict = sv.predict(x_test)
print("Accuracy score ",accuracy_score(y_test,y_predict))
print("\n Confusion metricx ")
print(confusion_matrix(y_test,y_predict))
print("\n classification Details ")
print(classification_report(y_test,y_predict))